In [1]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interpn
from scipy.ndimage import gaussian_filter

# Download the data from the following link:
# link = "https://weisslab.cs.ucl.ac.uk/WEISSTeaching/datasets/-/raw/promise12/image_train00.npy"
# !wget -q -O image_train00.npy $link

# Load the data using numpy and transpose it to match shape x, y, z
image_train00 = np.load("image_train00.npy").T
from Image3D import Image3D
# Experiment 1: Volume resizing
# Implement upsampling such that the resized volume has an isotropic voxel dimension
# An isotropic voxel dimension means that the size of the voxels in a 3D image is the same along all three dimensions.
# This can be useful in medical imaging, as it ensures that the image resolution is the same in all directions
# and allows for more accurate measurements.

# The voxel dimensions of a 3D image are directly related to the resolution of the image.
# In general, the smaller the voxel dimensions, the higher the resolution of the image.

# Create an instance of the Image3D class
# Voxel dimensions are set to 0.5, 0.5, 2 mm
image3d = Image3D(image_train00, (0.5, 0.5, 2))

# Create helper function for upsampling, downsampling and resampling images using Image3D class
def resize_image(image3d_object, final_voxel_dimension, antialias = False, sigma = 1):
    """
    This is a helper function for upsampling, downsampling and resampling images using Image3D class

    Parameters

    ----------
    image3d_object : Image3D
        An instance of the Image3D class
    final_voxel_dimension : float
        The final voxel dimension of the resized image
    antialias : bool
        If True, gaussian filtering is applied to the resized image
    sigma : float  
        The standard deviation of the gaussian filter

    Returns

    -------
    resized_image : numpy.ndarray
        The resized image

    """


    resize_ratio = np.array(image3d_object.voxel_dimension) / final_voxel_dimension
    if antialias:
        resized_image = image3d_object.volume_resize_antialias(resize_ratio, sigma = sigma)
    else:
        resized_image = image3d_object.volume_resize(resize_ratio)
    return resized_image

# Create function to save axial slices of the resized images
def save_axial_slices(original_image_object, resized_image_object, voxel_dimension, slice_number_origin, filename, title):
    """
    This is a helper function to save axial slices of the resized images

    Parameters

    ----------
    original_image_object : Image3D object
        The original image object
    resized_image : Image3D object
        The resized image object
    voxel_dimension : float
        The voxel dimension of the resized image
    slice_number_origin : int
        The slice number in the original image to be saved
    filename : str
        The filename of the saved image
    title : str
        The title of the saved image

    Returns

    -------
    None

    """

    ratio = int(original_image_object.voxel_dimension[2] / resized_image_object.voxel_dimension[2])
    fig, (ax) = plt.subplots(1, figsize=(6, 6))
    ax.imshow(resized_image_object.image[:, :, slice_number_origin*ratio], cmap = "bone")
    ax.set_title(f"Axial slice of {title}")
    #ax.set_title(f"Axial slice of upsampled image with voxel dimensions {voxel_dimension}mm")
    ax.set_xlabel("x (mm)")
    ax.set_ylabel("y (mm)")
    plt.savefig(f"{filename}_slice_{slice_number_origin}.png")
    plt.close(fig)


# ------------------------------------------------------------------------------------------------------------

# Experiment 1.1.1: Upsampling
# In upsampling, the final voxel dimension should be smaller than the original voxel dimension
# Time the execution of the upsampling function without pre-filtering
print(f"Time taken to upsample the image without pre-filtering \n")
%time
upsampled_image = resize_image(image3d_object = image3d, final_voxel_dimension = 0.4)

# save 5 axial slices as png images
# The axial slices should be saved in the following order: 15, 20, 22, 26, 31
for slice in [15, 20, 22, 26, 31]:
    save_axial_slices(original_image_object = image3d, resized_image_object = upsampled_image,
    voxel_dimension = upsampled_image.voxel_dimension,
    slice_number_origin = slice,
    filename = "exp1_upsampled", title= "upsampled image without pre-filter")



# Time the execution of the upsampling function with pre-filtering
print(f"Time taken to upsample the image with pre-filtering \n")
%time
filtered_upsampled_image = resize_image(image3d_object = image3d, final_voxel_dimension = 0.4, antialias = True, sigma = 1.5)

# save 5 axial slices as png images
# The axial slices should be saved in the following order: 15, 20, 22, 26, 31
for slice in [15, 20, 22, 26, 31]:
    save_axial_slices(original_image_object = image3d, resized_image_object = filtered_upsampled_image,
    voxel_dimension = filtered_upsampled_image.voxel_dimension,
    slice_number_origin = slice,
    filename = "exp1_upsampled_with_filter", title= "upsampled image with pre-filter")



# Print on the time taken to execute the upsampling
print(f"Time taken to upsample the image without pre-filtering is less than the time taken to upsample the image with pre-filtering. \n This is because the pre-filtering resizing is computationally expensive since a gaussian kernel is being slid over the entire image before resizing and interpolation is done.\n")


# ------------------------------------------------------------------------------------------------------

# Experiment 1.1.2: Downsampling
# In downsampling, the final voxel dimension should be larger than the original voxel dimension
# Time the execution of the downsampling function without pre-filtering
print(f"Time taken to downsample the image without pre-filtering \n")
%time
downsampled_image = resize_image(image3d_object = image3d, final_voxel_dimension = 1.0)

# save 5 axial slices as png images
# The axial slices should be saved in the following order: 15, 20, 22, 26, 31
for slice in [15, 20, 22, 26, 31]:
    save_axial_slices(original_image_object = image3d, resized_image_object = downsampled_image,
    voxel_dimension = downsampled_image.voxel_dimension,
    slice_number_origin = slice,
    filename = "exp1_downsampled", title= "downsampled image without pre-filter")

# Time the execution of the downsampling function with pre-filtering
print(f"Time taken to downsample the image with pre-filtering \n")
%time
filtered_downsampled_image = resize_image(image3d_object = image3d, final_voxel_dimension = 1.0, antialias = True, sigma = 1.5)

# save 5 axial slices as png images
# The axial slices should be saved in the following order: 15, 20, 22, 26, 31
for slice in [15, 20, 22, 26, 31]:
    save_axial_slices(original_image_object = image3d, resized_image_object = filtered_downsampled_image,
    voxel_dimension = filtered_downsampled_image.voxel_dimension,
    slice_number_origin = slice,
    filename = "exp1_downsampled_with_filter", title= "downsampled image with pre-filter")

# Print on the time taken to execute the downsampling
print(f"Time taken to downsample the image without pre-filtering is less than the time taken to downsample the image with pre-filtering. \n This is because the pre-filtering resizing is computationally expensive since a gaussian kernel is being slid over the entire image before resizing and interpolation is done.\n")


# ------------------------------------------------------------------------------------------------------

# Experiment 1.1.3: Resampling
# In resampling, the final voxel dimension can be either smaller or larger than the original voxel dimension
# Time the execution of the resampling function without pre-filtering
print(f"Time taken to resample the image without pre-filtering \n")
%time
resampled_image = resize_image(image3d_object = image3d, final_voxel_dimension = np.array(image3d.voxel_dimension) * 2)

# save 5 axial slices as png images
# The axial slices should be saved in the following order: 15, 20, 22, 26, 31
for slice in [15, 20, 22, 26, 31]:
    save_axial_slices(original_image_object = image3d, resized_image_object = resampled_image,
    voxel_dimension = resampled_image.voxel_dimension,
    slice_number_origin = slice,
    filename = "exp1_resampled", title= "resampled image without pre-filter")

# Time the execution of the resampling function
print(f"Time taken to resample the image with pre-filtering \n")
%time
filtered_resampled_image = resize_image(image3d_object = image3d, final_voxel_dimension = np.array(image3d.voxel_dimension) * 2, antialias = True, sigma = 1)

# save 5 axial slices as png images
# The axial slices should be saved in the following order: 15, 20, 22, 26, 31
for slice in [15, 20, 22, 26, 31]:
    save_axial_slices(original_image_object = image3d, resized_image_object = filtered_resampled_image,
    voxel_dimension = filtered_resampled_image.voxel_dimension,
    slice_number_origin = slice,
    filename = "exp1_resampled_with_filter", title= "resampled image with pre-filter")


# Print on the time taken to execute the resampling
print(f"Time taken to resample the image without pre-filtering is less than the time taken to resample the image with pre-filtering. \n This is because the pre-filtering resizing is computationally expensive since a gaussian kernel is being slid over the entire image before resizing and interpolation is done.\n")


Time taken to upsample the image without pre-filtering 

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs
Time taken to upsample the image with pre-filtering 

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.68 µs
Time taken to upsample the image without pre-filtering is less than the time taken to upsample the image with pre-filtering. 
 This is because the pre-filtering resizing is computationally expensive since a gaussian kernel is being slid over the entire image before resizing and interpolation is done.

Time taken to downsample the image without pre-filtering 

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 9.78 µs
Time taken to downsample the image with pre-filtering 

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs
Time taken to downsample the image without pre-filtering is less than the time taken to downsample the image with pre-filtering. 
 This is because the pre-filtering resizing is computationally expensive since a 